## Importing Modules

In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

import ValuationTool as vt

## Reading and Cleaning the Data

Using log prices to remove the outliers.
Removing the INDUS and AGE from the feature.

In [2]:
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICES'])

## Adding a Label to the Features

In [3]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = features.mean().values.transpose()


## Using Linear Regression

In [4]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

## Getting the Log Estimate with Upper and Lower Bound

Getting the user entered value for Number of Rooms and and PT Ratio. <br/>
Assigining mean value to everything else expect the user entered value.

In [5]:
def get_log_estimate(nr_room,students_per_classroom, next_to_river=False, high_confidence=True):

    property_stats[0][RM_IDX] = nr_room
    property_stats[0][PTRATIO_IDX] = students_per_classroom

    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0

    log_estimate = regr.predict(property_stats)[0][0]

    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68

    return log_estimate, upper_bound, lower_bound, interval



In [6]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE/ np.median(boston_dataset.target)

log_est, upper, lower, conf = vt.get_log_estimate(9, students_per_classroom=15, next_to_river=False, high_confidence=False)

dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f"The estimated property value is {rounded_est}.")
print(f"At {conf}% confidencethe valuation range is ")
print(f"USD {rounded_low} at the lower end to USD {rounded_hi} at the higher end")


The estimated property value is 827000.0.
At 68% confidencethe valuation range is 
USD 685000.0 at the lower end to USD 997000.0 at the higher end


In [7]:
def get_dollar_prices(rm, ptratio, chas=False, large_range=True):

    if rm < 1 or ptratio < 1 :
        print("This is unrealastic")
        return 

    log_est, upper, lower, conf = vt.get_log_estimate(rm, students_per_classroom=ptratio, next_to_river=chas, high_confidence=large_range)

    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f"The estimated property value is {rounded_est}.")
    print(f"At {conf}% confidencethe valuation range is, ")
    print(f"USD {rounded_low} at the lower end to USD {rounded_hi} at the higher end")

In [8]:
get_dollar_prices(rm=9, ptratio=30, large_range=False)

The estimated property value is 472000.0.
At 68% confidencethe valuation range is, 
USD 391000.0 at the lower end to USD 569000.0 at the higher end
